In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Jostein Barry-Straume
- Ernesto Carrera Ruvalcaba
- Adam Tschannen
- Kevin Cannon

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___
Enter your answer here:

- How many instances are in the dataset? 

> There are 11,314 instances in the dataset.

- What does each instance represent?

> Each instance represents a word index/array of 130,107 words that might be in the news document. Represents a sparse word vector.

- How many classes are in the dataset and what does each class represent?

> There are 130107 classes in the dataset, with each class representing how frequent a word appears in a given news document. There are 20 classes in the dataset.

- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?

> This document collection is 20 years old. Data is stale. English language, at least with regards to nouns, change at a rapid pace. 'Google' and 'Obama' were not in the vernacular 20 years ago.

- Is the data represented as a sparse or dense matrix?

> The data is represented as a sparse matrix. 12.14% of the data is non-zero.

___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [3]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:', euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:', cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a>0,b>0), 'ac:', jaccard(a>0,c>0), 'bc:', jaccard(b>0,c>0))

print('\n\nThe most appropriate distance is...')
print('Cosine. It clearly delineates between topics effectively. B and C are closest (by far)')
print('and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398234 bc: 0.9177794226661624
Cosine Distance
 ab: 0.603371411376 ac: 0.707027614956 bc: 0.421159534335
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.882113821138 ac: 0.875471698113 bc: 0.908794788274


The most appropriate distance is...
Cosine. It clearly delineates between topics effectively. B and C are closest (by far)
and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [4]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 
from sklearn.metrics import accuracy_score 
from sklearn import metrics as mt

cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K

max_accs = 0
max_k = 0

for k in range(1,10):
    clf = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric='euclidean')
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    acc = mt.accuracy_score(y_test,yhat)
    if max(max_accs, acc) == acc:
        max_accs = acc
        max_k = k
    print('Accuracy of', k, 'neighbors is', acc * 100, '%')

print('\nThe best accuracy is %f with %d neighbor(s)'%(max_accs,max_k))

/Users/atschannen/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/atschannen/anaconda3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Accuracy of 1 neighbors is 59.961110129 %
Accuracy of 2 neighbors is 54.4281421248 %
Accuracy of 3 neighbors is 52.8548700725 %
Accuracy of 4 neighbors is 49.9204525367 %
Accuracy of 5 neighbors is 48.9835601909 %
Accuracy of 6 neighbors is 48.0996994874 %
Accuracy of 7 neighbors is 47.268870426 %
Accuracy of 8 neighbors is 46.526427435 %
Accuracy of 9 neighbors is 45.7309528018 %

The best accuracy is 0.599611 with 1 neighbor(s)


**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:

> Easier to just calculate all the words. Sparse data won't branch well with so many frequent zero instances, so each tree will be very deep.


_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [5]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 
from sklearn.metrics import accuracy_score 
from sklearn import metrics as mt

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)


# fill in your code here

for d in ['l1','l2','cosine']:
    clf = NearestCentroid(metric=d)
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    acc = mt.accuracy_score(y_test,yhat)
    print('\nAccuracy of', d, 'metrics is', acc * 100, '%')    
    print('The best distance metric is: ', d)

/Users/atschannen/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "



Accuracy of l1 metrics is 30.9881562666 %
The best distance metric is:  l1

Accuracy of l2 metrics is 39.8444405162 %
The best distance metric is:  l2

Accuracy of cosine metrics is 47.5870602793 %
The best distance metric is:  cosine


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Enter you answer here:

> There are 2602140 conditionals probabilities that need to be parameterized. There are 20 priors that need to be parameterized.



In [6]:
# Use this space for any calculations you might want to do
#total conditionals = number of features * number of classes
total_conditionals = 20 * 130107
priors = 20
print('There are', total_conditionals, 'conditionals probabilities that need to be parameterized.')
print('There are', priors, 'priors that need to be parameterized.')

There are 2602140 conditionals probabilities that need to be parameterized.
There are 20 priors that need to be parameterized.


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

> Bernoulli uses binary/boolean attributes, whereas Multinomial Naive Bayes assumes various boolean categories. Both of these classifers are not appropriate for continuous attributes. Consequently, Gaussian Naive Bayes would be the most appropriate among the three options. It is the only one that incorporates continuous values into its function.

> If the data is sparse, this does not change the answer because Gaussian Naive Bayes is still the only one that deals with continuous attributes. Doesn't matter if the data is sparse or not, what matters is the fact that the data is continuous vs boolean/discrete.

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

alphas = [1, 0.1, 0.05, 0.01, 0.001]

for a in alphas:
    clf_mnb = MultinomialNB(alpha=a)
    clf_bnb = BernoulliNB(alpha=a, binarize=0.02)
    
    clf_mnb.fit(X_train, y_train)
    yhat1 = clf_mnb.predict(X_test)
    acc1 = mt.accuracy_score(y_test,yhat1)
    print('\n\nAt the alpha =', a, 'level...')
    print ('Accuracy for MultinomialNB is', acc1 * 100, '%')
    
    clf_bnb.fit(X_train, y_train)
    yhat2 = clf_bnb.predict(X_test)
    acc2 = mt.accuracy_score(y_test,yhat2)
    print ('Accuracy for BernoulliNB is', acc2 * 100, '%')

bins = [1, 0.1, 0.05, 0.02, 0.01, 0.001]

for b in bins:
    clf_mnb = MultinomialNB(alpha=0.001)
    clf_bnb = BernoulliNB(alpha=0.001, binarize=b)
    
    clf_mnb.fit(X_train, y_train)
    yhat1 = clf_mnb.predict(X_test)
    acc1 = mt.accuracy_score(y_test,yhat1)
    print('\n\nAt the binarize =', b, 'level...')
    print ('Accuracy for MultinomialNB is', acc1 * 100, '%')
    
    clf_bnb.fit(X_train, y_train)
    yhat2 = clf_bnb.predict(X_test)
    acc2 = mt.accuracy_score(y_test,yhat2)
    print ('Accuracy for BernoulliNB is', acc2 * 100, '%')   
            
print('\n\nThese classifiers are so fast because...',
      'both MultinomialNB and BernoulliNB are classifying on boolean attribute(s).')
print('Naive Bayes is faster because it takes a probabilistic approach.')
print('\n\nThe alpha values control...', 'the form of the model because alpha is a hyper, as well as smoothing, parameter.')




At the alpha = 1 level...
Accuracy for MultinomialNB is 69.5951917978 %
Accuracy for BernoulliNB is 70.655824642 %


At the alpha = 0.1 level...
Accuracy for MultinomialNB is 83.6662541983 %
Accuracy for BernoulliNB is 84.7445642567 %


At the alpha = 0.05 level...
Accuracy for MultinomialNB is 85.7168110306 %
Accuracy for BernoulliNB is 85.5930705321 %


At the alpha = 0.01 level...
Accuracy for MultinomialNB is 88.1562665724 %
Accuracy for BernoulliNB is 86.2471274527 %


At the alpha = 0.001 level...
Accuracy for MultinomialNB is 88.5805197101 %
Accuracy for BernoulliNB is 86.3531907371 %


At the binarize = 1 level...
Accuracy for MultinomialNB is 88.5805197101 %
Accuracy for BernoulliNB is 5.30316422132 %


At the binarize = 0.1 level...
Accuracy for MultinomialNB is 88.5805197101 %
Accuracy for BernoulliNB is 68.4815273113 %


At the binarize = 0.05 level...
Accuracy for MultinomialNB is 88.5805197101 %
Accuracy for BernoulliNB is 83.577868128 %


At the binarize = 0.02 level..

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.